# Loading and Checking data

In [1]:
\l E:\pycharm\PycharmProjects\kdbq\columbiaHdb\2019.01.03

In [2]:
\a
\c 5000 10000

,`bin10


In [3]:
bin10: update reverse fills reverse date,
     reverse fills reverse mid,
     reverse fills reverse spread,
     reverse fills reverse vol,
     reverse fills reverse adv
     by id from bin10
select  from bin10

time     id date       trade     mid      spread        vol         adv          
---------------------------------------------------------------------------------
09:30:00 0  2019.01.03 454071.8  91.715   0.001148717   0.000446367 1.122415e+008
09:30:10 0  2019.01.03 -171833.3 91.72671 0.001072136   0.000446367 1.122415e+008
09:30:20 0  2019.01.03 -301.2193 91.58853 7.658112e-005 0.000446367 1.122415e+008
09:30:30 0  2019.01.03 -78884.65 91.65646 0.000612649   0.000446367 1.122415e+008
09:30:40 0  2019.01.03 -24705.54 91.60259 0.0008423923  0.000446367 1.122415e+008
09:30:50 0  2019.01.03 -92166.19 91.58385 0.0005871219  0.000446367 1.122415e+008
09:31:00 0  2019.01.03 -22823.01 91.56276 0.000663703   0.000446367 1.122415e+008
09:31:10 0  2019.01.03 -4940.182 91.59088 0.0005615949  0.000446367 1.122415e+008
09:31:20 0  2019.01.03 -41691.88 91.59322 0.0005360678  0.000446367 1.122415e+008
09:31:30 0  2019.01.03 -539826.4 91.60727 0.0007402842  0.000446367 1.122415e+008
09:31:40 0  2019

# Exercise 1

## 1 TWAP Schedule

In [4]:
twap: {[trade;adv;size]
        time_ct_int : count(trade);
        traded_size : size%time_ct_int;
        time_ct_int#traded_size}

## 2 VWAP Schedule

In [5]:
vwap:{[trade;adv;size]
    vol_weight : abs(trade)%adv;
    vol_weight*size}

## 3 PoV Schedule

In [6]:
part_rate_float : 0.1
pov:{[trade;adv;size]
    deltas (sums part_rate_float*abs trade) & size
    }

## 4 Simulation with 3 schedules

In [7]:
size: 1000000
tb1_4: select twap_schedule : twap[trade;adv;size],
                    vwap_schedule : vwap[trade;adv;size],
                    pov_schedule : pov[trade;adv;size] from bin10 where id=0


In [8]:
tb1_4

twap_schedule vwap_schedule pov_schedule
----------------------------------------
427.1679      4045.49       45407.18    
427.1679      1530.925      17183.33    
427.1679      2.683672      30.12193    
427.1679      702.8119      7888.465    
427.1679      220.1106      2470.554    
427.1679      821.142       9216.619    
427.1679      203.3384      2282.301    
427.1679      44.01387      494.0182    
427.1679      371.448       4169.188    
427.1679      4809.508      53982.64    
427.1679      480.071       5388.389    
427.1679      2823.343      31689.62    
427.1679      78.11536      876.7784    
427.1679      1179.669      13240.78    
427.1679      463.7469      5205.164    
427.1679      3929.767      44108.28    
427.1679      1.626987      18.26154    
427.1679      37.38737      419.6414    
427.1679      1005.862      11289.95    
427.1679      454.7246      5103.896    
427.1679      182.1076      2044.002    
427.1679      1085.178      12180.2     
427.1679      21

## 5

In [9]:
tb1_5: select  vwap_schedule : vwap[trade;adv;0.1* adv],
                        pov_schedule : pov[trade;adv;0.1*adv] from bin10 where id=0

In [10]:
tb1_5

vwap_schedule pov_schedule
--------------------------
45407.18      45407.18    
17183.33      17183.33    
30.12193      30.12193    
7888.465      7888.465    
2470.554      2470.554    
9216.619      9216.619    
2282.301      2282.301    
494.0182      494.0182    
4169.188      4169.188    
53982.64      53982.64    
5388.389      5388.389    
31689.62      31689.62    
876.7784      876.7784    
13240.78      13240.78    
5205.164      5205.164    
44108.28      44108.28    
18.26154      18.26154    
419.6414      419.6414    
11289.95      11289.95    
5103.896      5103.896    
2044.002      2044.002    
12180.2       12180.2     
2428.728      2428.728    
1404.625      1404.625    
1228.333      1228.333    
10652.47      10652.47    
0             0           
22990.01      22990.01    
1815.989      1815.989    
2546.917      2546.917    
1878.133      1878.133    
302.6875      302.6875    
2125.342      2125.342    
3788.234      3788.234    
0             0           
3

The schedules given by the two strategies under perfect volume prediction are the same.

# Exercise 2

## 1 Computation of Impact adjusted price

In [11]:
/ Impact model with given parameter: lambda = 0.8, halflife = 60
computeImpact: {[trade;vol;adv;h] 
    dt:1;
    beta: (log 2)%h*6;
    dv:vol*sqrt (6*60*6.5);
    lamb:0.8;
    b:beta*dt;
    (neg count trade)#ema[b;][0,(lamb*dv*trade%adv)%b]}

In [12]:
tb2: update twap_trade: twap[trade;adv;0.1*adv] by id from bin10

In [13]:
tb2

time     id date       trade     mid      spread        vol         adv           twap_trade
--------------------------------------------------------------------------------------------
09:30:00 0  2019.01.03 454071.8  91.715   0.001148717   0.000446367 1.122415e+008 4794.596  
09:30:10 0  2019.01.03 -171833.3 91.72671 0.001072136   0.000446367 1.122415e+008 4794.596  
09:30:20 0  2019.01.03 -301.2193 91.58853 7.658112e-005 0.000446367 1.122415e+008 4794.596  
09:30:30 0  2019.01.03 -78884.65 91.65646 0.000612649   0.000446367 1.122415e+008 4794.596  
09:30:40 0  2019.01.03 -24705.54 91.60259 0.0008423923  0.000446367 1.122415e+008 4794.596  
09:30:50 0  2019.01.03 -92166.19 91.58385 0.0005871219  0.000446367 1.122415e+008 4794.596  
09:31:00 0  2019.01.03 -22823.01 91.56276 0.000663703   0.000446367 1.122415e+008 4794.596  
09:31:10 0  2019.01.03 -4940.182 91.59088 0.0005615949  0.000446367 1.122415e+008 4794.596  
09:31:20 0  2019.01.03 -41691.88 91.59322 0.0005360678  0.000446367 1.

In [14]:
tb2_1: update twap_impact:computeImpact[twap_trade;vol;adv;360] by id from tb2

tb2_1: update Impact_adjusted_price: mid-twap_impact by id from tb2_1

tb2_1

time     id date       trade     mid      spread        vol         adv           twap_trade twap_impact   Impact_adjusted_price
--------------------------------------------------------------------------------------------------------------------------------
09:30:00 0  2019.01.03 454071.8  91.715   0.001148717   0.000446367 1.122415e+008 4794.596   7.378848e-007 91.715               
09:30:10 0  2019.01.03 -171833.3 91.72671 0.001072136   0.000446367 1.122415e+008 4794.596   1.475533e-006 91.7267              
09:30:20 0  2019.01.03 -301.2193 91.58853 7.658112e-005 0.000446367 1.122415e+008 4794.596   2.212944e-006 91.58853             
09:30:30 0  2019.01.03 -78884.65 91.65646 0.000612649   0.000446367 1.122415e+008 4794.596   2.950119e-006 91.65646             
09:30:40 0  2019.01.03 -24705.54 91.60259 0.0008423923  0.000446367 1.122415e+008 4794.596   3.687057e-006 91.60259             
09:30:50 0  2019.01.03 -92166.19 91.58385 0.0005871219  0.000446367 1.122415e+008 4794.596   4.42

## 2 New price with twap schedule impact

In [15]:
tb2_2:update twap_20_trade: twap[trade;adv;0.2*adv] by id from tb2_1
tb2_2:update twap_20_impact: computeImpact[twap_20_trade;vol;adv;10] by id from tb2_2
tb2_2 :update twap_price: twap_20_impact+Impact_adjusted_price from tb2_2

In [16]:
tb2_2

time     id date       trade     mid      spread        vol         adv           twap_trade twap_impact   Impact_adjusted_price twap_20_trade twap_20_impact twap_price
------------------------------------------------------------------------------------------------------------------------------------------------------------------------
09:30:00 0  2019.01.03 454071.8  91.715   0.001148717   0.000446367 1.122415e+008 4794.596   7.378848e-007 91.715                9589.191      1.47577e-006   91.715    
09:30:10 0  2019.01.03 -171833.3 91.72671 0.001072136   0.000446367 1.122415e+008 4794.596   1.475533e-006 91.7267               9589.191      2.93449e-006   91.72671  
09:30:20 0  2019.01.03 -301.2193 91.58853 7.658112e-005 0.000446367 1.122415e+008 4794.596   2.212944e-006 91.58853              9589.191      4.376359e-006  91.58854  
09:30:30 0  2019.01.03 -78884.65 91.65646 0.000612649   0.000446367 1.122415e+008 4794.596   2.950119e-006 91.65646              9589.191      5.801571e-00

## 3 New price with vwap schedule impact

In [17]:
tb2_3:update vwap_10_trade:vwap[trade;adv;0.1*adv] by id from tb2_2
tb2_3:update vwap_impact: computeImpact[vwap_10_trade;vol;adv;10] by id from tb2_3
tb2_3:update vwap_price: vwap_impact+Impact_adjusted_price from tb2_3

In [18]:
tb2_3

time     id date       trade     mid      spread        vol         adv           twap_trade twap_impact   Impact_adjusted_price twap_20_trade twap_20_impact twap_price vwap_10_trade vwap_impact   vwap_price
---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
09:30:00 0  2019.01.03 454071.8  91.715   0.001148717   0.000446367 1.122415e+008 4794.596   7.378848e-007 91.715                9589.191      1.47577e-006   91.715     45407.18      6.988132e-006 91.71501  
09:30:10 0  2019.01.03 -171833.3 91.72671 0.001072136   0.000446367 1.122415e+008 4794.596   1.475533e-006 91.7267               9589.191      2.93449e-006   91.72671   17183.33      9.551904e-006 91.72671  
09:30:20 0  2019.01.03 -301.2193 91.58853 7.658112e-005 0.000446367 1.122415e+008 4794.596   2.212944e-006 91.58853              9589.191      4.376359e-006  91.58854  

## 4 historical strategy's average impact costs

In [19]:
deltas_forward:{
            a:first[x] -': x;
            a:-1 xprev a;
            0^a}

In [20]:
a: 1 2 3 4 5
deltas_forward[a]

1 1 1 1 0


In [21]:
tb2_4: select hist_impact_cost:sum (twap_impact+0.5*deltas_forward[twap_impact])*twap_trade,
                        twap_impact_cost: sum  (twap_20_impact+0.5*deltas_forward[twap_20_impact])*twap_20_trade,
                        vwap_impact_cost:sum(vwap_impact+0.5*deltas_forward[vwap_impact])*vwap_10_trade
            by id from tb2_3
tb2_4

id | hist_impact_cost twap_impact_cost vwap_impact_cost
---| --------------------------------------------------
0  | 7668.624         2763.455         904.1144        
1  | 1978.024         712.7982         198.5323        
2  | 1024.739         369.2736         102.7037        
3  | 852.0666         307.0496         104.1368        
4  | 1132.069         407.9507         135.8458        
5  | 627.1745         226.0078         66.42819        
6  | 1303.165         469.607          125.6514        
7  | 641.4139         231.1391         62.51639        
8  | 3959.959         1427.006         481.8671        
9  | 1900.886         685.0009         179.8314        
10 | 9438.425         3401.218         916.9518        
11 | 3434.623         1237.696         339.2243        
12 | 1755.779         632.71           211.0721        
13 | 1124.076         405.0705         119.9891        
14 | 77343.31         27871.33         7425.591        
15 | 4164.194         1500.603         413.2935 

In [22]:
tb2_4_2: select hist_impact_cost: avg hist_impact_cost,
                        twap_impact_cost: avg twap_impact_cost,
                        vwap_impact_cost: avg vwap_impact_cost
            from tb2_4

In [23]:
tb2_4_2

hist_impact_cost twap_impact_cost vwap_impact_cost
--------------------------------------------------
2877.206         1036.826         297.4276        


## 5 historical strategy's average impact costs

In [24]:
tb2_5: select hist_pnl:sum ((sums twap_trade) * deltas_forward[Impact_adjusted_price])-(twap_impact+0.5*deltas_forward[twap_impact])*twap_trade,
                        twap_pnl: sum ((sums twap_20_trade) * deltas_forward[Impact_adjusted_price])- (twap_20_impact+0.5*deltas_forward[twap_20_impact])*twap_20_trade,
                        vwap_pnl:sum ((sums vwap_10_trade) * deltas_forward[Impact_adjusted_price])-(vwap_impact+0.5*deltas_forward[vwap_impact])*vwap_10_trade
            by id from tb2_3
tb2_5:select hist_pnl: avg hist_pnl,twap_pnl:avg twap_pnl,vwap_pnl:avg vwap_pnl from tb2_5

In [25]:
tb2_5

hist_pnl       twap_pnl       vwap_pnl
--------------------------------------
-1.004362e+007 -2.008252e+007 -9048384
